In [1]:
# import Libraries
import pandas as pd
import numpy as np
import matplotlib as pyplot
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# to see all the comands result in a single kernal 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# to increase no. of rows and column visibility in outputs
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 2000)

#To ignore warnings
import warnings
warnings.simplefilter('ignore')

In [2]:
train_df = pd.read_csv("Train.csv", parse_dates = ["InvoiceDate"])
print(train_df.head())

train_df_org = pd.read_csv("Train.csv", parse_dates = ["InvoiceDate"])

# Converting datetime to date objects.
train_df['InvoiceDate_org'] = train_df['InvoiceDate']
train_df['InvoiceDate'] = train_df['InvoiceDate'].apply(lambda x:x.date())
train_df['CustomerID'] = train_df['CustomerID'].apply(lambda x:str(int(float(x))))

train_df_org['CustomerID'] = train_df['CustomerID'].apply(lambda x:str(int(float(x))))

train_df['Quantity_org'] = train_df['Quantity']
train_df['Quantity'] = train_df['Quantity'].apply(abs)

train_df['Return_Flag'] = np.where(train_df['Quantity_org']<0,1,0)

train_df['Key'] = range(1, len(train_df)+1, 1)


# Train DF for negative values only and for Description

train_df_neg = train_df.copy()

print(len(train_df))
train_df = train_df[train_df['Quantity_org']>0]
train_df = train_df[train_df['UnitPrice']>0]
print(len(train_df))

#train_df.head()




   InvoiceNo  StockCode  Description  Quantity         InvoiceDate  UnitPrice  CustomerID  Country
0       6141       1583          144         3 2011-05-06 16:54:00       3.75     14056.0       35
1       6349       1300         3682         6 2011-05-11 07:35:00       1.95     13098.0       35
2      16783       2178         1939         4 2011-11-20 13:20:00       5.95     15044.0       35
3      16971       2115         2983         1 2011-11-22 12:07:00       0.83     15525.0       35
4       6080       1210         2886        12 2011-05-06 09:00:00       1.65     13952.0       35
284780
278579


In [3]:
# Checking correlation
#plt.figure(figsize=(15, 8))
#sns.heatmap(train_df.corr(),annot=True)

In [4]:
# Target varibale distribution 
#train_df['UnitPrice'].plot(kind = 'density', title = 'Price Distribution')

In [5]:
"""

# Analysing distribution in Categorical varibales 
cat_cols = ['StockCode', 'Description', 'InvoiceDate', 'CustomerID', 'Country']
fig, axes = plt.subplots(1,6, figsize=(24, 10))

for i, c in enumerate(cat_cols):
    _ = train_df[c].value_counts()[::-1].plot(kind = 'pie', ax=axes[i], title=c, autopct='%.0f', fontsize=18)
    _ = axes[i].set_ylabel('')
    
_ = plt.tight_layout()

"""

"\n\n# Analysing distribution in Categorical varibales \ncat_cols = ['StockCode', 'Description', 'InvoiceDate', 'CustomerID', 'Country']\nfig, axes = plt.subplots(1,6, figsize=(24, 10))\n\nfor i, c in enumerate(cat_cols):\n    _ = train_df[c].value_counts()[::-1].plot(kind = 'pie', ax=axes[i], title=c, autopct='%.0f', fontsize=18)\n    _ = axes[i].set_ylabel('')\n    \n_ = plt.tight_layout()\n\n"

In [6]:
#Checking relation between quantity and Target

#sns.jointplot(x=train_df['Quantity'], y=train_df['UnitPrice'])

In [7]:
test_df = pd.read_csv("Test.csv", parse_dates = ["InvoiceDate"])

# Converting datetime to date objects.
test_df['InvoiceDate_org'] = test_df['InvoiceDate']
test_df['InvoiceDate'] = test_df['InvoiceDate'].apply(lambda x:x.date())
test_df['CustomerID'] = test_df['CustomerID'].apply(lambda x:str(int(float(x))))
test_df['Quantity_org'] = test_df['Quantity']

test_df['Return_Flag'] = np.where(test_df['Quantity_org']<0,1,0)
test_df['Quantity'] = test_df['Quantity'].apply(abs)
#print(test_df.head())

test_df['Key'] = range(1, len(test_df)+1, 1)



In [8]:
# Combining Test and Train and creating new Days since first sold var at Desc level

train_df['is_train'] = 1
test_df['is_train'] = 0
train_df_neg['is_Train'] = 1

df = pd.concat([train_df,test_df],ignore_index=True)
df_neg = pd.concat([train_df_neg,test_df],ignore_index=True)

df_first_date_desc = df.groupby('Description', as_index = False).agg({ 'InvoiceDate': 'min'}).\
                                    rename(columns = {'InvoiceDate': 'Min_Date_Desc'})

df_neg_first_date_desc = df_neg.groupby('Description', as_index = False).agg({ 'InvoiceDate': 'min'}).\
                                    rename(columns = {'InvoiceDate': 'Min_Date_Desc'})

df_first_date_stock = df.groupby(['Description', 'StockCode'], as_index = False).agg({ 'InvoiceDate': 'min'}).\
                                    rename(columns = {'InvoiceDate': 'Min_Date_Stock'})

df = df.merge(df_first_date_desc, on = ['Description'], how = 'left')
df = df.merge(df_first_date_stock, on = ['Description', 'StockCode'], how = 'left')
df_neg = df_neg.merge(df_neg_first_date_desc, on = ['Description'], how = 'left')

df['days_since_desc'] = df.apply(lambda x : (x['InvoiceDate'] - x['Min_Date_Desc']).days, axis =1)
df['days_since_stock'] = df.apply(lambda x : (x['InvoiceDate'] - x['Min_Date_Stock']).days, axis =1)

df_neg['days_since_desc'] = df_neg.apply(lambda x : (x['InvoiceDate'] - x['Min_Date_Desc']).days, axis =1)

#df.head()

In [9]:
#Checking relation between days_since_desc and Target
#train_data = df[df['is_train']==1]
#sns.jointplot(x=train_data['days_since_desc'], y=train_data['UnitPrice'])

In [10]:
# Encoding Calendar Date

min_date = df['InvoiceDate'].min()
df['days_since_calendar'] = df.apply(lambda x : (x['InvoiceDate'] - min_date).days, axis =1)

#df['days_since_calendar'].head()

In [11]:
""" Addding New Variable CountOfOrders, CountOfDistinctDate, NoOfUniquePricePoints, NoOfUniqueCustomers, NoOfCountries,
Country35 Flag for each row
Desc level -> Price min,max,median,mode,mean,var
Desc level -> Quantity min,max,median,mode,mean,var
"""

df['NoOfOrders'] = df.groupby(['Description'])['InvoiceNo'].transform('count')
df['CountOfDistinctDate'] = df.groupby(['Description'])['InvoiceDate'].transform(pd.Series.nunique)
df['NoOfUniquePricePoints'] = df.groupby(['Description'])['UnitPrice'].transform(pd.Series.nunique)
df['NoOfUniqueCustomers'] = df.groupby(['Description'])['CustomerID'].transform(pd.Series.nunique)
df['NoOfCountries'] = df.groupby(['Description'])['Country'].transform(pd.Series.nunique)
df['NoOfUniqueStocks'] = df.groupby(['Description'])['StockCode'].transform(pd.Series.nunique)
df['Country35'] = np.where(df['Country']==35,1,0)
df['DataAvailable'] = df.groupby(['Description'])['days_since_desc'].transform('max')

df_neg['Country35'] = np.where(df_neg['Country']==35,1,0)

print(len(train_df))
print(len(test_df))
print(len(df))


' Addding New Variable CountOfOrders, CountOfDistinctDate, NoOfUniquePricePoints, NoOfUniqueCustomers, NoOfCountries,\nCountry35 Flag for each row\nDesc level -> Price min,max,median,mode,mean,var\nDesc level -> Quantity min,max,median,mode,mean,var\n'

278579
122049
400628


In [12]:
""" Addding New Variable for
Desc level -> Price min,max,median,mode,mean,var
Desc level -> Quantity min,max,median,mode,mean,var
"""

df['Price_Mean'] = df.groupby(['Description'])['UnitPrice'].transform('mean')
df['Price_Median'] = df.groupby(['Description'])['UnitPrice'].transform('median')
df['Price_Max'] = df.groupby(['Description'])['UnitPrice'].transform('max')
df['Price_Min'] = df.groupby(['Description'])['UnitPrice'].transform('min')
df['Price_StDev'] = df.groupby(['Description'])['UnitPrice'].transform('std')

df['Price_Mean_SC'] = df.groupby(['StockCode'])['UnitPrice'].transform('mean')
df['Price_Median_SC'] = df.groupby(['StockCode'])['UnitPrice'].transform('median')
df['Price_Max_SC'] = df.groupby(['StockCode'])['UnitPrice'].transform('max')
df['Price_Min_SC'] = df.groupby(['StockCode'])['UnitPrice'].transform('min')
df['Price_StDev_SC'] = df.groupby(['StockCode'])['UnitPrice'].transform('std')

df['Price_Mean_Cust'] = df.groupby(['CustomerID'])['UnitPrice'].transform('mean')
df['Price_Median_Cust'] = df.groupby(['CustomerID'])['UnitPrice'].transform('median')
df['Price_Max_Cust'] = df.groupby(['CustomerID'])['UnitPrice'].transform('max')
df['Price_Min_Cust'] = df.groupby(['CustomerID'])['UnitPrice'].transform('min')
df['Price_StDev_Cust'] = df.groupby(['CustomerID'])['UnitPrice'].transform('std')

df['Quantity_Mean_Cust'] = df.groupby(['CustomerID'])['Quantity'].transform('mean')
df['Quantity_Median_Cust'] = df.groupby(['CustomerID'])['Quantity'].transform('median')
df['Quantity_Max_Cust'] = df.groupby(['CustomerID'])['Quantity'].transform('max')
df['Quantity_Min_Cust'] = df.groupby(['CustomerID'])['Quantity'].transform('min')
df['Quantity_StDev_Cust'] = df.groupby(['CustomerID'])['Quantity'].transform('std')

df['Price_Mean_Country'] = df.groupby(['Country'])['UnitPrice'].transform('mean')
df['Price_Median_Country'] = df.groupby(['Country'])['UnitPrice'].transform('median')
df['Price_Max_Country'] = df.groupby(['Country'])['UnitPrice'].transform('max')
df['Price_Min_Country'] = df.groupby(['Country'])['UnitPrice'].transform('min')
df['Price_StDev_Country'] = df.groupby(['Country'])['UnitPrice'].transform('std')

df['Price_Mean_DescCoutry'] = df.groupby(['Country'])['UnitPrice'].transform('mean')
df['Price_Median_DescCoutry'] = df.groupby(['Country'])['UnitPrice'].transform('median')
df['Price_Max_DescCoutry'] = df.groupby(['Country'])['UnitPrice'].transform('max')
df['Price_Min_DescCoutry'] = df.groupby(['Country'])['UnitPrice'].transform('min')
df['Price_StDev_DescCoutry'] = df.groupby(['Country'])['UnitPrice'].transform('std')


df['Quantity_Mean'] = df.groupby(['Description'])['Quantity'].transform('mean')
df['Quantity_Median'] = df.groupby(['Description'])['Quantity'].transform('median')
df['Quantity_Max'] = df.groupby(['Description'])['Quantity'].transform('max')
df['Quantity_Min'] = df.groupby(['Description'])['Quantity'].transform('min')
df['Quantity_StDev'] = df.groupby(['Description'])['Quantity'].transform('std')


# Total Buy Of Customer in Train and Total Quantity 



' Addding New Variable for\nDesc level -> Price min,max,median,mode,mean,var\nDesc level -> Quantity min,max,median,mode,mean,var\n'

In [13]:
""" Addding New Variable for
Desc level -> Price min,max,median,mode,mean,var
Desc level -> Quantity min,max,median,mode,mean,var
"""

df_neg['Price_Mean'] = df_neg.groupby(['Description'])['UnitPrice'].transform('mean')
df_neg['Price_Median'] = df_neg.groupby(['Description'])['UnitPrice'].transform('median')
df_neg['Price_Max'] = df_neg.groupby(['Description'])['UnitPrice'].transform('max')
df_neg['Price_Min'] = df_neg.groupby(['Description'])['UnitPrice'].transform('min')
df_neg['Price_StDev'] = df_neg.groupby(['Description'])['UnitPrice'].transform('std')

df_neg['Price_Mean_SC'] = df_neg.groupby(['StockCode'])['UnitPrice'].transform('mean')
df_neg['Price_Median_SC'] = df_neg.groupby(['StockCode'])['UnitPrice'].transform('median')
df_neg['Price_Max_SC'] = df_neg.groupby(['StockCode'])['UnitPrice'].transform('max')
df_neg['Price_Min_SC'] = df_neg.groupby(['StockCode'])['UnitPrice'].transform('min')
df_neg['Price_StDev_SC'] = df_neg.groupby(['StockCode'])['UnitPrice'].transform('std')

df_neg['Price_Mean_Cust'] = df_neg.groupby(['CustomerID'])['UnitPrice'].transform('mean')
df_neg['Price_Median_Cust'] = df_neg.groupby(['CustomerID'])['UnitPrice'].transform('median')
df_neg['Price_Max_Cust'] = df_neg.groupby(['CustomerID'])['UnitPrice'].transform('max')
df_neg['Price_Min_Cust'] = df_neg.groupby(['CustomerID'])['UnitPrice'].transform('min')
df_neg['Price_StDev_Cust'] = df_neg.groupby(['CustomerID'])['UnitPrice'].transform('std')

df_neg['Quantity_Mean_Cust'] = df_neg.groupby(['CustomerID'])['Quantity'].transform('mean')
df_neg['Quantity_Median_Cust'] = df_neg.groupby(['CustomerID'])['Quantity'].transform('median')
df_neg['Quantity_Max_Cust'] = df_neg.groupby(['CustomerID'])['Quantity'].transform('max')
df_neg['Quantity_Min_Cust'] = df_neg.groupby(['CustomerID'])['Quantity'].transform('min')
df_neg['Quantity_StDev_Cust'] = df_neg.groupby(['CustomerID'])['Quantity'].transform('std')

df_neg['Price_Mean_Country'] = df_neg.groupby(['Country'])['UnitPrice'].transform('mean')
df_neg['Price_Median_Country'] = df_neg.groupby(['Country'])['UnitPrice'].transform('median')
df_neg['Price_Max_Country'] = df_neg.groupby(['Country'])['UnitPrice'].transform('max')
df_neg['Price_Min_Country'] = df_neg.groupby(['Country'])['UnitPrice'].transform('min')
df_neg['Price_StDev_Country'] = df_neg.groupby(['Country'])['UnitPrice'].transform('std')

df_neg['Price_Mean_DescCoutry'] = df_neg.groupby(['Country'])['UnitPrice'].transform('mean')
df_neg['Price_Median_DescCoutry'] = df_neg.groupby(['Country'])['UnitPrice'].transform('median')
df_neg['Price_Max_DescCoutry'] = df_neg.groupby(['Country'])['UnitPrice'].transform('max')
df_neg['Price_Min_DescCoutry'] = df_neg.groupby(['Country'])['UnitPrice'].transform('min')
df_neg['Price_StDev_DescCoutry'] = df_neg.groupby(['Country'])['UnitPrice'].transform('std')


df_neg['Quantity_Mean'] = df_neg.groupby(['Description'])['Quantity'].transform('mean')
df_neg['Quantity_Median'] = df_neg.groupby(['Description'])['Quantity'].transform('median')
df_neg['Quantity_Max'] = df_neg.groupby(['Description'])['Quantity'].transform('max')
df_neg['Quantity_Min'] = df_neg.groupby(['Description'])['Quantity'].transform('min')
df_neg['Quantity_StDev'] = df_neg.groupby(['Description'])['Quantity'].transform('std')


# Total Buy Of Customer in Train and Total Quantity 



' Addding New Variable for\nDesc level -> Price min,max,median,mode,mean,var\nDesc level -> Quantity min,max,median,mode,mean,var\n'

In [14]:
print(len(df))
print(len(df_neg))

400628
406829


In [15]:
# Add Mode Values  for Price and Quantity

def getMode1(x):
    return x.value_counts().index[0]

def getMode2(x):
    if len(x.value_counts())>1:
        return x.value_counts().index[1]
    else:
        return x.value_counts().index[0]

train_agg = train_df.groupby(['Description'], as_index = False).agg(\
                                            Price_Mode1 = ('UnitPrice', getMode1),\
                                            Price_Mode2 = ('UnitPrice', getMode2),\
                                            Quantity_Mode = ('Quantity', getMode1))

df = df.merge(train_agg, on = ['Description'], how = 'left')

In [16]:
#Checking relation between Description and Price_Mean
#train_data = df[df['is_train']==1]
#sns.jointplot(x=train_data['Description'], y=train_data['Price_Mean'])

In [17]:
# Create Flag for bad Customers

bad_customer_list = ['14096','17450','14298','14088']

df['bad_customer_flag'] = np.where(df['CustomerID'].isin(bad_customer_list),1,0)
df_neg['bad_customer_flag'] = np.where(df_neg['CustomerID'].isin(bad_customer_list),1,0)

#df.to_csv('Combined_Df_with_vars.csv', index = False)

In [18]:

#df.head()


In [19]:
# Create Personal Encoder on Min Max value

#desc_train1 = train_df['Price_Max', 'Price_Min', 'Price_Mode'].drop_duplicates()

df['Price_Mode_Sum'] = df['Price_Mode1'] + df['Price_Mode2']


In [20]:
"""
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object 
onehotencoder = OneHotEncoder()

encoder = OneHotEncoder(sparse=False)
train_X_encoded = pd.DataFrame (encoder.fit_transform(df[['Country']]))
train_X_encoded.columns = encoder.get_feature_names(['Country'])

df = pd.concat([df, train_X_encoded], axis=1)
#droping the country column 
#df= df.drop(['Country'], axis=1) 
#printing to verify 
df.head()"""

"\nfrom sklearn.preprocessing import OneHotEncoder\n# creating one hot encoder object \nonehotencoder = OneHotEncoder()\n\nencoder = OneHotEncoder(sparse=False)\ntrain_X_encoded = pd.DataFrame (encoder.fit_transform(df[['Country']]))\ntrain_X_encoded.columns = encoder.get_feature_names(['Country'])\n\ndf = pd.concat([df, train_X_encoded], axis=1)\n#droping the country column \n#df= df.drop(['Country'], axis=1) \n#printing to verify \ndf.head()"

In [21]:
#df[encoder.get_feature_names(['Country'])].sum()

#encoder.get_feature_names(['Country'])

In [22]:

train_df = df[df['is_train']==1]
test_df = df[df['is_train']==0]
train_df_neg = df_neg[df_neg['is_Train']==1]

print(len(train_df_neg))
print(len(train_df))
print(len(test_df))

#desc_train2 = train_df['Price_Max', 'Price_Min', 'Price_Mode'].drop_duplicates()

#prine(len(dist_train))

#df.to_csv('df_with_all_vals.csv', index = False)

284780
278579
122049


In [23]:
#train_data = df[df['is_train']==1]
#sns.jointplot(x=train_df['Price_Mode_Sum'], y=train_df['UnitPrice'])

In [24]:
# Category Encoding based on target data

"""
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder


feature_list = ['Description']
#target = 'UnitPrice'
target = 'UnitPrice'


CBE_encoder = CatBoostEncoder()
train_cbe = CBE_encoder.fit_transform(train_df[feature_list], train_df[target])
test_cbe = CBE_encoder.transform(test_df[feature_list])

train_df['NewDescription'] = train_cbe

print(train_df['NewDescription'].nunique())
print(train_df['Description'].nunique())

train_df.to_csv('Encoded_Train.csv', index = False)



T_encoder = MEstimateEncoder()
train_cbe = T_encoder.fit_transform(train_df[feature_list], train_df[target])
test_cbe = T_encoder.transform(test_df[feature_list])
"""

"""
MEE_encoder = MEstimateEncoder()
train_mee = MEE_encoder.fit_transform(train[feature_list], target)
test_mee = MEE_encoder.transform(test[feature_list])
"""


"\nfrom category_encoders.ordinal import OrdinalEncoder\nfrom category_encoders.woe import WOEEncoder\nfrom category_encoders.target_encoder import TargetEncoder\nfrom category_encoders.sum_coding import SumEncoder\nfrom category_encoders.m_estimate import MEstimateEncoder\nfrom category_encoders.leave_one_out import LeaveOneOutEncoder\nfrom category_encoders.helmert import HelmertEncoder\nfrom category_encoders.cat_boost import CatBoostEncoder\nfrom category_encoders.james_stein import JamesSteinEncoder\nfrom category_encoders.one_hot import OneHotEncoder\n\n\nfeature_list = ['Description']\n#target = 'UnitPrice'\ntarget = 'UnitPrice'\n\n\nCBE_encoder = CatBoostEncoder()\ntrain_cbe = CBE_encoder.fit_transform(train_df[feature_list], train_df[target])\ntest_cbe = CBE_encoder.transform(test_df[feature_list])\n\ntrain_df['NewDescription'] = train_cbe\n\nprint(train_df['NewDescription'].nunique())\nprint(train_df['Description'].nunique())\n\ntrain_df.to_csv('Encoded_Train.csv', index = Fa

'\nMEE_encoder = MEstimateEncoder()\ntrain_mee = MEE_encoder.fit_transform(train[feature_list], target)\ntest_mee = MEE_encoder.transform(test[feature_list])\n'

# MODEL FITTING

In [25]:
#Importing Packages
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV,KFold,StratifiedKFold
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn import preprocessing



In [26]:
def run_gradient_boosting(clf,k, fit_params, train, test, features):
    N_SPLITS = k
    oofs = np.zeros(len(train))
    preds = np.zeros((len(test)))

    folds = KFold(n_splits = N_SPLITS,random_state=42)

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train[TARGET_COL])):
        print(f'\n------------- Fold {fold_ + 1} -------------')
        target = train[TARGET_COL]
        X_trn, y_trn = train[features].iloc[trn_idx], target.iloc[trn_idx]

        X_val, y_val = train[features].iloc[val_idx], target.iloc[val_idx]

        X_test = test[features]

        scaler = StandardScaler()
        #scaler = preprocessing.MinMaxScaler()
        #scaler = preprocessing.RobustScaler()
        _ = scaler.fit(X_trn)

        X_trn = scaler.transform(X_trn)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

        _ = clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], **fit_params)
        preds_val = clf.predict(X_val)
        preds_test = clf.predict(X_test)
        #preds_test1 = (np.exp(preds_test)-1)
        
        fold_score =  np.sqrt(mean_squared_error(y_val,preds_val)) 
        print(f'\n RMSE score for validation set is {fold_score}')

        oofs[val_idx] = preds_val
        preds += preds_test / N_SPLITS


    oofs_score = np.sqrt(mean_squared_error(target, oofs))
    print(f'\n\n rmse score for oofs is {oofs_score}')
    return oofs, preds, oofs_score

In [27]:
#All cols in current data
print(df.columns)

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'InvoiceDate_org', 'Quantity_org', 'Return_Flag', 'Key', 'is_train', 'Min_Date_Desc', 'Min_Date_Stock', 'days_since_desc', 'days_since_stock', 'days_since_calendar', 'NoOfOrders', 'CountOfDistinctDate', 'NoOfUniquePricePoints', 'NoOfUniqueCustomers', 'NoOfCountries', 'NoOfUniqueStocks', 'Country35', 'DataAvailable', 'Price_Mean', 'Price_Median', 'Price_Max', 'Price_Min', 'Price_StDev', 'Price_Mean_SC', 'Price_Median_SC', 'Price_Max_SC', 'Price_Min_SC', 'Price_StDev_SC', 'Price_Mean_Cust', 'Price_Median_Cust', 'Price_Max_Cust', 'Price_Min_Cust', 'Price_StDev_Cust', 'Quantity_Mean_Cust', 'Quantity_Median_Cust', 'Quantity_Max_Cust', 'Quantity_Min_Cust', 'Quantity_StDev_Cust', 'Price_Mean_Country', 'Price_Median_Country', 'Price_Max_Country', 'Price_Min_Country', 'Price_StDev_Country', 'Price_Mean_DescCoutry', 'Price_Median_DescCoutry', 'Price_Max_DescCoutry', 'Price_Min_DescCou

In [28]:
# Selecting Columns to take in the model
model_col = [ \
            'Price_Mean','Price_Median' ,'Price_Max', 'Price_Min', 'Price_StDev','Quantity',\
             'days_since_desc','Price_Mode1','Country35',\
             #'days_since_calendar'
             #'Price_Mean_Country', 'Price_Median_Country', 'Price_Max_Country', 'Price_Min_Country', 'Price_StDev_Country',\
             #'Quantity_Median_Cust',\
             'Quantity_Mean','Quantity_Median', 'Quantity_Max', 'Quantity_Min', 'Quantity_StDev', 'Quantity_Mode',\
             'Price_Mean_Cust', 'Price_Median_Cust', 'Price_Max_Cust', 'Price_Min_Cust', 'Price_StDev_Cust',\
             'Price_Mean_DescCoutry', 'Price_Median_DescCoutry', 'Price_Max_DescCoutry', 'Price_Min_DescCoutry',\
             'Price_StDev_DescCoutry'
            #'bad_customer_flag'\
            ]


cat_col = ['Description', 'StockCode']
tar_col = ['UnitPrice']

In [29]:
#test_df['Key'] = range(1, len(test_df)+1, 1)
#train_df['Key'] = range(1, len(train_df)+1, 1)


In [30]:
desc_list = [2624, 2140]

print(len(train_df))
print(len(test_df))

#test_df.to_csv('Testiiing.csv', index = False)

#
#train_df1 = train_df[((train_df['bad_customer_flag'] == 0) & ~(train_df['Description'].isin(desc_list)))]
#test_df1 = test_df[((test_df['bad_customer_flag'] == 0) & ~(test_df['Description'].isin(desc_list)))]

train_df1 = train_df[(train_df['bad_customer_flag'] == 0) ]
test_df1 = test_df[(test_df['bad_customer_flag'] == 0) ]


#
train_df2 = train_df[train_df['bad_customer_flag'] == 1]
test_df2 = test_df[test_df['bad_customer_flag'] == 1]


print(len(train_df1))
print(len(test_df1))


278579
122049
273208
119734


In [31]:


xgb = XGBRegressor(n_estimators = 10000,
#                        learning_rate = 0.05,
#                       colsample_bytree = 0.76,
                        )
fit_params = {'verbose': False, 'early_stopping_rounds': 100}
TARGET_COL= 'UnitPrice'
features = model_col
xgb_oofs, xgb_preds,score_xgb = run_gradient_boosting(xgb,10, fit_params, train_df1, test_df1, features)

train_df1['y_xgb'] = xgb_oofs
test_df1['y_xgb'] = xgb_preds


------------- Fold 1 -------------

 RMSE score for validation set is 5.974283329530361

------------- Fold 2 -------------

 RMSE score for validation set is 3.6491043916425343

------------- Fold 3 -------------

 RMSE score for validation set is 14.072228046600305

------------- Fold 4 -------------

 RMSE score for validation set is 3.3724309790901574

------------- Fold 5 -------------

 RMSE score for validation set is 4.24791324688471

------------- Fold 6 -------------

 RMSE score for validation set is 1.6171446159015563

------------- Fold 7 -------------

 RMSE score for validation set is 1.8946169946689098

------------- Fold 8 -------------

 RMSE score for validation set is 8.848543313262526

------------- Fold 9 -------------

 RMSE score for validation set is 7.129191243318177

------------- Fold 10 -------------

 RMSE score for validation set is 5.343175955791143


 rmse score for oofs is 6.63573920707743


In [32]:

lgb = LGBMRegressor(n_estimators = 10000, importance_type='gain',
#                          learning_rate = 0.05,
#                          colsample_bytree = 0.76,
                        )
fit_params = {'verbose': False, 'early_stopping_rounds': 100}
TARGET_COL= 'UnitPrice'
features = model_col
lgb_oofs, lgb_preds, score_lgb = run_gradient_boosting(lgb,10, fit_params, train_df1, test_df1, features)

train_df1['y_lgb'] = lgb_oofs
test_df1['y_lgb'] = lgb_preds



------------- Fold 1 -------------

 RMSE score for validation set is 5.924738418025408

------------- Fold 2 -------------

 RMSE score for validation set is 3.995389782213259

------------- Fold 3 -------------

 RMSE score for validation set is 13.164928029330326

------------- Fold 4 -------------

 RMSE score for validation set is 4.789895208065252

------------- Fold 5 -------------

 RMSE score for validation set is 3.1068305506701273

------------- Fold 6 -------------

 RMSE score for validation set is 1.9873594159411676

------------- Fold 7 -------------

 RMSE score for validation set is 2.848835179963958

------------- Fold 8 -------------

 RMSE score for validation set is 6.571711174288212

------------- Fold 9 -------------

 RMSE score for validation set is 4.028852705005997

------------- Fold 10 -------------

 RMSE score for validation set is 3.5872038236722346


 rmse score for oofs is 5.842349296904673


In [33]:
cb = CatBoostRegressor(iterations=10000,
                       learning_rate = 0.1,
#                         colsample_bytree = 0.76,
                        )

fit_params = {'verbose': False, 'early_stopping_rounds': 100}
TARGET_COL= 'UnitPrice'

#train_df1['CustomerIDA'] = train_df1['CustomerID']+'A'
#test_df1['CustomerIDA'] = test_df1['CustomerID']+'A'

features = model_col

cb_oofs, cb_preds, scores_cb = run_gradient_boosting(cb,10, fit_params, train_df1, test_df1, features)

train_df1['y_cb'] = cb_oofs
test_df1['y_cb'] = cb_preds



------------- Fold 1 -------------

 RMSE score for validation set is 3.256334865114697

------------- Fold 2 -------------

 RMSE score for validation set is 1.2050423804510115

------------- Fold 3 -------------

 RMSE score for validation set is 12.294222985189249

------------- Fold 4 -------------

 RMSE score for validation set is 2.7702326709048393

------------- Fold 5 -------------

 RMSE score for validation set is 1.480320712035936

------------- Fold 6 -------------

 RMSE score for validation set is 1.5807461327563748

------------- Fold 7 -------------

 RMSE score for validation set is 1.8816086539204855

------------- Fold 8 -------------

 RMSE score for validation set is 5.981248238435534

------------- Fold 9 -------------

 RMSE score for validation set is 4.336102596070534

------------- Fold 10 -------------

 RMSE score for validation set is 2.8976517218306617


 rmse score for oofs is 4.9201909411474825


In [34]:
# Here You can even extract your training data and analyse on which outcomes you are getting most error
# and take further step accordingly

print('XGB', np.sqrt(mean_squared_error(train_df1['UnitPrice'], train_df1['y_xgb'])))
print('LGB', np.sqrt(mean_squared_error(train_df1['UnitPrice'], train_df1['y_lgb'])))
print('CB', np.sqrt(mean_squared_error(train_df1['UnitPrice'], train_df1['y_cb'])))

desc_list = [2624,2140]
#desc_list = [2140]

train_ch = train_df1[~(train_df1['Description'].isin( desc_list))]

print('XGB', np.sqrt(mean_squared_error(train_ch['UnitPrice'], train_ch['y_xgb'])))
print('LGB', np.sqrt(mean_squared_error(train_ch['UnitPrice'], train_ch['y_lgb'])))
print('CB', np.sqrt(mean_squared_error(train_ch['UnitPrice'], train_ch['y_cb'])))

#vis = train[model_col]
#train_df1.to_csv('train_out.csv',index=False)


#Initial
#XGB 6.75708061980588
#LGB 6.8533119642410485
#CB 6.192579534217409
#XGB 1.374504379844585
#LGB 2.214419022731006
#CB 1.6746977994468253


#With cust variables only
#XGB 6.62136934556261
#LGB 6.0595596619508765
#CB 5.090020090736837
#XGB 2.9104907925809838
#LGB 1.9568225605270688
#CB 0.8598545791541393

#With countrydesc and without cust variables
#XGB 7.5361980307975465
#LGB 6.414495187642012
#CB 6.62048569699907
#XGB 3.36982995579794
#LGB 2.170495846545204
#CB 2.0324233072277753

#with countrydesc and cust variables
#All
#XGB 6.63573920707743
#LGB 5.842349296904673
#CB 4.9201909411474825
#Remove Descriptions
#XGB 2.8537366457337447
#LGB 1.990223335117116
#CB 0.8572806379070284

# With CountryPrice Var
#XGB 6.667124219826937
#LGB 5.842349296904673
#CB 5.14830477284481
#XGB 2.923244780114524
#LGB 1.990223335117116
#CB 0.9715364217698965

#With StockCode
#XGB 6.585061062531734
#LGB 5.904042617902602
#CB 5.271007264281459
#XGB 2.810480890310904
#LGB 2.007563133657509
#CB 0.9835345193264253

#With StockCode Mean only
#XGB 6.5814832385682545
#LGB 5.90072246520115
#CB 5.1718583089414185
#XGB 2.807175281711653
#LGB 2.0142463106278146
#CB 1.0822050726769732

# With CustomerQuantity
#XGB 6.413276923221273
#LGB 5.656222028715484
#CB 5.220617493020439
#XGB 2.8378301012186107
#LGB 1.980336931615718
#CB 0.8874526123657804

# With Customer Quantity Mode
#XGB 6.67639498035905
#LGB 5.634793975234537
#CB 5.001626564245285
#XGB 2.8310387757103124
#LGB 1.9829330418853484
#CB 0.9743152626497211


# Lets go with CB

XGB 6.63573920707743
LGB 5.842349296904673
CB 4.9201909411474825
XGB 2.8537366457337447
LGB 1.990223335117116
CB 0.8572806379070284


In [35]:
train_df1['y_merge'] = 0.1*train_df1['y_xgb'] + 0.1*train_df1['y_lgb']+ 0.8*train_df1['y_cb']
test_df1['y_merge'] = 0.1*test_df1['y_xgb'] + 0.1*test_df1['y_lgb']+ 0.8*test_df1['y_cb']
np.sqrt(mean_squared_error(train_df1['UnitPrice'], train_df1['y_merge']))

5.027236272179873

In [47]:
# Create Model Only for Bad Descriptions


desc_list = [2624]
#desc_list = [2140]

train_df_desc_2624 = train_df_neg[train_df_neg['Description'].isin(desc_list)]
test_df_desc_2624 = test_df[test_df['Description'].isin(desc_list)]

# Selecting Columns to take in the model
model_col = [ \
            'Price_Mean','Price_Median' ,'Price_Max', 'Price_Min', 'Price_StDev','Quantity',\
             'days_since_desc','Country35','Return_Flag',\
             #'days_since_calendar'
             #'Price_Mean_Country', 'Price_Median_Country', 'Price_Max_Country', 'Price_Min_Country', 'Price_StDev_Country',\
             #'Quantity_Median_Cust',\
             'Quantity_Mean','Quantity_Median', 'Quantity_Max', 'Quantity_Min', 'Quantity_StDev',\
             'Price_Mean_Cust', 'Price_Median_Cust', 'Price_Max_Cust', 'Price_Min_Cust', 'Price_StDev_Cust',\
             'Price_Mean_DescCoutry', 'Price_Median_DescCoutry', 'Price_Max_DescCoutry', 'Price_Min_DescCoutry',\
             'Price_StDev_DescCoutry'
            #'bad_customer_flag'\
            ]

cat_col = ['Description', 'StockCode']
tar_col = ['UnitPrice']

xgb = XGBRegressor(n_estimators = 1000,
#                        learning_rate = 0.05,
#                       colsample_bytree = 0.76,
                        )
fit_params = {'verbose': False, 'early_stopping_rounds': 100}
TARGET_COL= 'UnitPrice'
features = model_col

print('---------------XGBooster-----------------')
xgb_oofs, xgb_preds,score_xgb = run_gradient_boosting(xgb,10, fit_params, train_df_desc_2624, test_df_desc_2624, features)

cb = CatBoostRegressor(iterations=10000,
                       learning_rate = 0.1,
#                         colsample_bytree = 0.76,
                        )

print('---------------CatBooster-----------------')
cb_oofs, cb_preds, scores_cb = run_gradient_boosting(cb,10, fit_params, train_df_desc_2624, test_df_desc_2624, features)

train_df_desc_2624['y_xgb'] = xgb_oofs
test_df_desc_2624['y_xgb'] = xgb_preds

train_df_desc_2624['y_cb'] = cb_oofs
test_df_desc_2624['y_cb'] = cb_preds


test_df_desc_2624.to_csv('test_bad_desc.csv', index = False)
train_df_desc_2624.to_csv('train_bad_desc.csv',index=False)

---------------XGBooster-----------------

------------- Fold 1 -------------

 RMSE score for validation set is 19.921509829761504

------------- Fold 2 -------------

 RMSE score for validation set is 8.700223382011949

------------- Fold 3 -------------

 RMSE score for validation set is 4.681825164446201

------------- Fold 4 -------------

 RMSE score for validation set is 39.41147728890589

------------- Fold 5 -------------

 RMSE score for validation set is 7.804104021127562

------------- Fold 6 -------------

 RMSE score for validation set is 7.059205030821113

------------- Fold 7 -------------

 RMSE score for validation set is 23.666303599342445

------------- Fold 8 -------------

 RMSE score for validation set is 18.184841517548758

------------- Fold 9 -------------

 RMSE score for validation set is 881.2707625360179

------------- Fold 10 -------------

 RMSE score for validation set is 57.826873964956796


 rmse score for oofs is 279.82697645606765
---------------Cat

In [48]:
#train_df1.columns
#train_df_desc_2624.columns


In [49]:
# Combining with old values and Check the error Afterwards

#train_er = train_df1[~(train_df1['Description'].isin( desc_list))]

print('XGB', np.sqrt(mean_squared_error(train_df_desc_2624['UnitPrice'], train_df_desc_2624['y_xgb'])))
print('CB', np.sqrt(mean_squared_error(train_df_desc_2624['UnitPrice'], train_df_desc_2624['y_cb'])))


test_df_desc_2624_fm = test_df_desc_2624[['Key','y_xgb', 'y_cb']].rename(columns = {'y_xgb':'y_xgb_baddesc',\
                                                                                    'y_cb':'y_cb_baddesc'})

train_df1_merged = train_df1.merge(test_df_desc_2624_fm, on = 'Key', how = 'left')

train_df1_merged['y_xgb_merged'] = np.where(train_df1_merged['y_xgb_baddesc'].isna(),train_df1_merged['y_xgb']\
                                            ,train_df1_merged['y_xgb_baddesc'])
train_df1_merged['y_cb_merged'] = np.where(train_df1_merged['y_cb_baddesc'].isna(),train_df1_merged['y_cb']\
                                            ,train_df1_merged['y_cb_baddesc'])


print('XGB', np.sqrt(mean_squared_error(train_df1_merged['UnitPrice'], train_df1_merged['y_xgb'])))
print('CB', np.sqrt(mean_squared_error(train_df1_merged['UnitPrice'], train_df1_merged['y_cb'])))

#vis = train[model_col]
#train_df1.to_csv('train_out.csv',index=False)

XGB 279.82697645606765
CB 279.1588758375569
XGB 6.63573920707743
CB 4.9201909411474825


In [ ]:
#Bad Customer data

score_df_badcust_xgb = pd.DataFrame()
train_final_badcust_xgb = pd.DataFrame()
test_final_badcust_xgb = pd.DataFrame()

i = 0
for desc,train_d1 in train_df[train_df['bad_customer_flag'] == 1].groupby(['CustomerID','Description']):
    #print('----------------------',desc,'----------------------')
    xgb = XGBRegressor(n_estimators = 100,
    #                        learning_rate = 0.05,
    #                       colsample_bytree = 0.76,
                            )
    
    fit_params = {'verbose': False, 'early_stopping_rounds': 100}
    TARGET_COL= 'UnitPrice'
    features = model_col
    test_d1 = test_df[(test_df['Description'] == desc[1]) & (test_df['CustomerID'] == desc[0])]
    split_val = min(len(train_d1),10)
    
    if len(test_d1)>0 and len(train_d1)>1:
        xgb_oofs, xgb_preds, score = run_gradient_boosting(xgb, split_val, fit_params, train_d1, test_d1, features)
        
    elif len(train_d1) == 1:
        un_val =  list(train_d1['UnitPrice'].unique())[0]
        xgb_oofs, xgb_preds, score = np.zeros(len(train_d1)), np.repeat(un_val,len(test_d1)), 0
        
    else:
        xgb_oofs, xgb_preds, score = np.zeros(len(train_d1)), np.zeros(len(test_d1)), 0
        
    temp_df = pd.DataFrame({'Description':[desc[1]], 'CustomerID':[desc[0]] ,'Price_Mode':[desc], 'xgb_score':[score],\
                            'Length_of_train':[len(train_d1)],\
                            'Length_of_test':[len(test_d1)],'unique_Desc':[train_d1['Description'].nunique()]})
    
    score_df_badcust_xgb = score_df_badcust_xgb.append(temp_df)
    
    # Adding in train data
    # Adding in test data
    train_d1['y_xgb'] = xgb_oofs
    test_d1['y_xgb'] = xgb_preds
    train_final_badcust_xgb = train_final_badcust_xgb.append(train_d1)
    test_final_badcust_xgb = test_final_badcust_xgb.append(test_d1)
    i += 1
    #if i>=10:
    #    break
        

score_df_badcust_xgb.to_csv('xgb_10_fold_score_bad_cust.csv', index = False)
train_final_badcust_xgb.to_csv('xgb_10_fold_train_bad_cust.csv', index = False)
test_final_badcust_xgb.to_csv('xgb_10_fold_test_bad_cust.csv', index = False)




In [ ]:
# One Price Point and No Of Samples >=10 then override with Mode Price for all test values

print(len(test_df))
print(len(test_df1))
print(len(test_final_badcust_xgb))

print(test_df1.columns)
#print(test_df1.columns)

test_merged = test_df.merge(test_df1[['Key', 'y_cb']], on = 'Key', how = 'left')

#test_merged = test_merged.merge(test_final_lgb[['Key', 'y_lgb']], on = 'Key', how = 'left')

# Combining with customer data
test_merged = test_merged.merge(test_final_badcust_xgb[['Key', 'y_xgb']].rename(columns = {'y_xgb':'y_badcust'}),\
                                on = 'Key', how = 'left')

test_merged['Comb_Price_with_bad_cust'] = np.where(test_merged['y_badcust'].isna(),\
                                    test_merged['y_cb'], test_merged['y_badcust'])

test_merged['Comb_Price_with_org'] = np.where(((test_merged['NoOfUniquePricePoints'] == 1) &\
                                     (test_merged['DataAvailable'] > 100)),\
                                    test_merged['Price_Mode1'], test_merged['Comb_Price_with_bad_cust'])

#test_merged['Comb_Price_with_bad_desc'] = np.where(test_merged['Description'] == 2140,\
#                                    test_merged['Manual_Value'], test_merged['Comb_Price_with_org'])

#test_merged['Comb_Price_with_bad_desc'] = np.where(test_merged['Description'] == 2624,\
#                                    test_merged['y_merge'], test_merged['Comb_Price_with_bad_desc'])


# Create Customer Mean Df to fill for IDS with No Description
Customer_Mean_DF = train_df.groupby(['CustomerID'], as_index = False).agg(cust_mean_price= ('UnitPrice','mean'))
test_merged = test_merged.merge(Customer_Mean_DF, on = 'CustomerID', how = 'left')


test_merged['y_final1'] = np.where( test_merged['Comb_Price_with_bad_cust'].isna() ,\
                                    test_merged['cust_mean_price'], test_merged['Comb_Price_with_org'])


# Join with Original Df at Description, Day Quantity to fill actual values if available

left_on = ['InvoiceNo', 'StockCode', 'Description', 'Quantity_org', 'InvoiceDate_org', 'CustomerID', 'Country']
right_on = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'CustomerID', 'Country']

train_unique = train_df_org.groupby(right_on, as_index = False).agg(OrgPrice = ('UnitPrice', 'last'))
#print(train_unique.head())

test_merged = test_merged.merge(train_unique
                                , left_on = left_on, right_on = right_on, how = 'left')

#print(test_merged.columns)

test_merged['y_final2'] = np.where( test_merged['OrgPrice'].isna() ,\
                                    test_merged['y_final1'], test_merged['OrgPrice'])


#test_merged.to_csv('submission_type1.csv',index = False)


In [ ]:
# you can change predictions to xgb_pred and lgb_pred to extract their outputs

test_merged['UnitPrice'] = test_merged['y_final1'].apply(abs)
test_merged[['UnitPrice']].to_csv('my_submission_file.csv', index=False)



# END

In [ ]:
#Bad Description

desc_list = [2624,2140]
#desc_list = [2140]

score_df_badcust_xgb = pd.DataFrame()
train_final_badcust_xgb = pd.DataFrame()
test_final_badcust_xgb = pd.DataFrame()

i = 0
for country,train_d1 in train_df[train_df['Description'].isin( desc_list)].groupby(['Country']):
    print('----------------------',country,'----------------------')
    xgb = XGBRegressor(n_estimators = 100,
    #                        learning_rate = 0.05,
    #                       colsample_bytree = 0.76,
                            )
    
    fit_params = {'verbose': False, 'early_stopping_rounds': 100}
    TARGET_COL= 'UnitPrice'
    features = model_col
    test_d1 = test_df[(test_df['Description'] == desc_list[i]) & (test_df['Country'] == country)]
    split_val = min(len(train_d1),10)
    
    if len(test_d1)>0 and len(train_d1)>1:
        xgb_oofs, xgb_preds, score = run_gradient_boosting(xgb, split_val, fit_params, train_d1, test_d1, features)
        
    elif len(train_d1) == 1:
        un_val =  list(train_d1['UnitPrice'].unique())[0]
        xgb_oofs, xgb_preds, score = np.zeros(len(train_d1)), np.repeat(un_val,len(test_d1)), 0
        
    else:
        xgb_oofs, xgb_preds, score = np.zeros(len(train_d1)), np.zeros(len(test_d1)), 0
        
    temp_df = pd.DataFrame({'Description':[desc[1]], 'CustomerID':[desc[0]] ,'Price_Mode':[desc], 'xgb_score':[score],\
                            'Length_of_train':[len(train_d1)],\
                            'Length_of_test':[len(test_d1)],'unique_Desc':[train_d1['Description'].nunique()]})
    
    score_df_badcust_xgb = score_df_badcust_xgb.append(temp_df)
    
    # Adding in train data
    # Adding in test data
    train_d1['y_xgb'] = xgb_oofs
    test_d1['y_xgb'] = xgb_preds
    train_final_badcust_xgb = train_final_badcust_xgb.append(train_d1)
    test_final_badcust_xgb = test_final_badcust_xgb.append(test_d1)
    i += 1
    #if i>=10:
    #    break
        

score_df_badcust_xgb.to_csv('xgb_10_fold_score_bad_cust.csv', index = False)
train_final_badcust_xgb.to_csv('xgb_10_fold_train_bad_cust.csv', index = False)
test_final_badcust_xgb.to_csv('xgb_10_fold_test_bad_cust.csv', index = False)




In [ ]:
#desc_list = [2624]
#desc_list = [2140]

#train_df_desc = train_df[train_df['Description'].isin(desc_list)]
#test_df_desc = test_df[test_df['Description'].isin(desc_list)]


#sns.jointplot(x=train_df_desc['days_since_desc'], y=train_df_desc['UnitPrice'])

In [ ]:
# Overwriting For Desc 2624

train_df_desc_2624['y_merge'] = 0.2*train_df_desc_2624['y_xgb'] + 0.8*train_df_desc_2624['y_cb']
np.sqrt(mean_squared_error(train_df_desc_2624['UnitPrice'], train_df_desc_2624['y_merge']))

test_df_desc_2624['y_merge'] = 0.2*train_df_desc_2624['y_xgb'] + 0.8*train_df_desc_2624['y_cb']

test_df_desc_2624.at[((test_df_desc_2624['InvoiceNo'] == 5882 )\
                      &(test_df_desc_2624['days_since_desc'] == 153 ))\
                     ,'y_merge'] = 1

test_df_desc_2624.at[((test_df_desc_2624['InvoiceNo'] == 5922 )\
                      &(test_df_desc_2624['days_since_desc'] == 153 ))\
                     ,'y_merge'] = 8142.75

test_df_desc_2624.at[((test_df_desc_2624['InvoiceNo'] == 5876 )\
                      &(test_df_desc_2624['days_since_desc'] == 153 ))\
                     ,'y_merge'] = 1



In [ ]:
# Remove Bad Customers and create Bad Customers Model seperate

#train_df1
#train_df2

"""

# Selecting Columns to take in the model
model_col = [ 'CustomerID','Price_Mode_Sum',\
            'Price_Mean','Price_Median' ,'Price_Max', 'Price_Min', 'Price_StDev','Quantity',\
             'days_since_desc',\
           #'Quantity_Mean','Quantity_Median', 'Quantity_Max', 'Quantity_Min', 'Quantity_StDev', 'Quantity_Mode'\
            #'bad_customer_flag'\
            ]
cat_col = ['Description', 'StockCode']
tar_col = ['UnitPrice']

xgb = XGBRegressor(n_estimators = 1000,
#                        learning_rate = 0.05,
#                       colsample_bytree = 0.76,
                        )
fit_params = {'verbose': False, 'early_stopping_rounds': 100}
TARGET_COL= 'UnitPrice'
features = model_col
xgb_oofs, xgb_preds,score_xgb = run_gradient_boosting(xgb,10, fit_params, train_df2, test_df2, features)

train_df2['y_xgb'] = xgb_oofs
test_df2['y_xgb'] = xgb_preds

"""

In [ ]:
# Handling Data For Bad Description 2140 and 2624

test_desc_2140 = pd.read_csv('Manual_Entry.csv', parse_dates = ['InvoiceDate'])
test_desc_2140['CustomerID'] = test_desc_2140['CustomerID'].apply(lambda x:str(int(float(x))))

#test_desc_2140.head()
#test_df.head()

#print(test_desc_2140.columns)
#print(test_df.columns)

# Manual_Value
left_on = ['InvoiceNo', 'StockCode', 'Description', 'Quantity_org', 'InvoiceDate_org', 'CustomerID', 'Country']
right_on = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'CustomerID', 'Country']

test_merged = test_df.merge(test_desc_2140, left_on = left_on, right_on = right_on, how = 'left')

# For 2624
test_merged = test_merged.merge(test_df_desc_2624[['Key', 'y_merge']], on = 'Key', how = 'left')

test_merged.to_csv('Test_Merged.csv', index = False)

In [ ]:
# One Price Point and No Of Samples >=10 then override with Mode Price for all test values

'NoOfOrders', 'CountOfDistinctDate', 'NoOfUniquePricePoints'

print(len(test_df))
print(len(test_df1))
print(len(test_final_badcust_xgb))

print(test_df1.columns)
#print(test_df1.columns)

#test_df1['y_merge'] = 0.45*test_df1['y_xgb'] + 0.1*test_df1['y_xgb'] + 0.45*test_df1['y_xgb']

test_merged = test_merged.merge(test_df1[['Key', 'l_xgb']], on = 'Key', how = 'left')

#test_merged = test_merged.merge(test_final_lgb[['Key', 'y_lgb']], on = 'Key', how = 'left')

# Combining with customer data
test_merged = test_merged.merge(test_final_badcust_xgb[['Key', 'y_xgb']].rename(columns = {'y_xgb':'y_badcust'}),\
                                on = 'Key', how = 'left')

test_merged['Comb_Price_with_bad_cust'] = np.where(test_merged['y_badcust'].isna(),\
                                    test_merged['y_xgb'], test_merged['y_badcust'])

test_merged['Comb_Price_with_bad_desc'] = np.where(test_merged['Description'] == 2140,\
                                    test_merged['Manual_Value'], test_merged['Comb_Price_with_bad_cust'])

test_merged['Comb_Price_with_bad_desc'] = np.where(test_merged['Description'] == 2624,\
                                    test_merged['y_merge'], test_merged['Comb_Price_with_bad_desc'])


# Create Customer Mean Df to fill for IDS with No Description
Customer_Mean_DF = train_df.groupby(['CustomerID'], as_index = False).agg(cust_mean_price= ('UnitPrice','mean'))
test_merged = test_merged.merge(Customer_Mean_DF, on = 'CustomerID', how = 'left')


test_merged['y_final1'] = np.where( test_merged['Comb_Price_with_bad_desc'].isna() ,\
                                    test_merged['cust_mean_price'], test_merged['Comb_Price_with_bad_desc'])

# Join with Original Df at Description, Day Quantity to fill actual values if available

left_on = ['InvoiceNo', 'StockCode', 'Description', 'Quantity_org', 'InvoiceDate_org', 'CustomerID', 'Country']
right_on = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'CustomerID', 'Country']

train_unique = train_df_org.groupby(right_on, as_index = False).agg(OrgPrice = ('UnitPrice', 'mean'))
print(train_unique.head())

test_merged = test_merged.merge(train_unique
                                , left_on = left_on, right_on = right_on, how = 'left')

print(test_merged.columns)

test_merged['y_final2'] = np.where( test_merged['OrgPrice'].isna() ,\
                                    test_merged['y_final1'], test_merged['OrgPrice'])


test_merged.to_csv('submission_type1.csv',index = False)


In [ ]:
# you can change predictions to xgb_pred and lgb_pred to extract their outputs

test_merged['UnitPrice'] = test_merged['y_final1'].apply(abs)
test_merged[['UnitPrice']].to_csv('my_submission_file.csv', index=False)

